In [ ]:
import maestroflame

In [ ]:
# this is the path to your data files
data_path = '../data/flame/'

# These is the input/output prefix of your datafile names.
input_prefix = 'react_inputs_*'
output_prefix = 'react_outputs_*'

# Plotfile prefixes, used for visualization purposes.
plotfile_prefix = 'flame_*'

# By default, this package will save your model, logs of the training and testing data during training,
# and plots to a directory. Here you specify that directory.
output_dir = 'testing123/'

# The log file. Everything that is printed during training also goes into this file in case something
# gets interrupted.
log_file = output_dir + "log.txt"

In [ ]:
# We first remove the directory we just generated. If you don't do this you'll get an error. This is to protect
# this package from overwriting your data in case one forgets to change the output_dir when training a new model

!rm -r testing123/

In [ ]:
from maestroflame.train import NuclearReactionML
nrml = NuclearReactionML(data_path, input_prefix, output_prefix, plotfile_prefix,
                output_dir, log_file, DEBUG_MODE=False, DO_PLOTTING=True,
                SAVE_MODEL=True, DO_HYPER_OPTIMIZATION=False)

In [ ]:
# the package provides preset up loss functions and networks to use
from maestroflame.losses import log_loss, log_loss_2
from maestroflame.networks import Net
import torch.optim as optim
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

num_epochs = 200
model = Net(16, 16, 16, 16, 14)

# get model to cuda if possible
model.to(device=device)
optimizer = optim.Adam(model.parameters(), lr=1e-5)

nrml.train(model, optimizer, num_epochs, log_loss_2)

In [ ]:
# need to put model on cpu for plotting
model.to(device=torch.device("cpu"))

nrml.plot()

In [ ]:
import glob
plots = glob.glob('testing123/*.png')
from IPython.display import Image, display

for plot in plots:
    fig = Image(filename=(plot))
    display(fig)

In [ ]:
data, targets = next(iter(nrml.train_loader))
print(f"Data batch shape: {data.size()}")
print(f"Targets batch shape: {targets.size()}")

In [ ]:
model.to(device=torch.device("cpu"))
pred = model(data)
print(f"Prediction batch shape: {pred.size()}")
print(f"Targets batch shape: {targets.size()}")

In [ ]:
import yt
import numpy as np
ds = yt.load("../data/flame/react_inputs_0000010_1")
dt = ds.current_time.to_value()
yloc = yt.YTArray(2.8, 'cm')
# print(ds.r[0.3:0.35,0:yloc])
ad = ds.r[0.3:0.31, 0:yloc]
for i,field in enumerate(ds._field_list):
    if i == 0:
        data = np.zeros([len(ds._field_list), len(ad[field])])
        data[i,:] = np.array(ad[field])
print(data.shape)

In [ ]:
import yt
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

fields = [field for field in yt.load("../data/flame/react_outputs_0000010_1")._field_list]
# print(fields)

nnuc = len(fields)-1
colors = matplotlib.cm.rainbow(np.linspace(0, 1, nnuc+1))
plt.clf()
with torch.no_grad():
    for i in range(pred.shape[0]):
        if i == 0:
            for j in range(pred.shape[1]):
                plt.scatter(pred[i,j], targets[i,j], 
                                color=colors[j], label=fields[j])
        else:
            plt.scatter(pred[i, :], targets[i,:nnuc+1], c=colors)
    plt.plot(np.linspace(0, 1), np.linspace(0,1), '--', color='orange')
    plt.legend(bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
x = np.array([1,2,3,4])
i = np.where(x<0)[0]
print(i)
x = [0, 3]+list(range(5,13))
print(x)

In [ ]:
def my_heaviside(x, input):
    y = torch.zeros_like(x)
    y[x < 0] = 0
    y[x > 0] = 1
    y[x == 0] = input
    return y

In [ ]:
import torch.nn as nn
import torch

def log_loss_spec(prediction, target):
    # Log Loss Function for all species except C12[1], O16[2], Mg24[4]
    # If there are negative values in X we use MSE
    # Enuc stays with MSE because its normalized

    # species that use linear loss
    spec_lin = [1, 2, 4]
    # species that use log loss
    spec_log = [0, 3] + list(range(5,13))

    # X is not allowed to be negative. Enuc is
    X_lin, X_log = prediction[:, spec_lin], prediction[:, spec_log]
    X_lin_target, X_log_target = target[:, spec_lin], target[:, spec_log]
    enuc = prediction[:, 13]
    enuc_target = target[:, 13]

    L = nn.MSELoss()

    enuc_loss = L(enuc, enuc_target)


    # we want to penalize negative numbers for
    # species that use linear loss
    if torch.sum(X_lin < 0) > 0:
        # how much do we hate negative numbers?
        factor = 1000 # a lot

        value = torch.tensor([0.], device=device)

        # greater than zero we apply mse loss
        A = my_heaviside(X_lin, value)

        # less than zero we apply large mse loss
        B = -my_heaviside(X_lin, value) + 1

        linear_loss =  torch.sum(A * L(X_lin, X_lin_target) + factor * B * L(X_lin, X_lin_target))

    else:
        linear_loss = L(X_lin, X_lin_target)


    # if there are negative numbers we cant use log on mass fractions
    if torch.sum(X_log < 0) > 0:
        # how much do we hate negative log numbers?
        factor = 10e6 # a lot a lot

        log_loss = factor * L(X_log, X_log_target)

    else:
        log_loss = torch.abs(.01*L(torch.log(X_log), torch.log(X_log_target)))


    return enuc_loss + linear_loss + log_loss


In [ ]:
spec_lin = [1, 2, 4]
spec_log = [0, 3] + list(range(5,13))

X_lin, X_log = pred[:, spec_lin], pred[:, spec_log]
X_lin_target, X_log_target = targets[:, spec_lin], targets[:, spec_log]

A = my_heaviside(X_log, torch.tensor([0.]))
B = -my_heaviside(X_log, torch.tensor([0.])) + 1
print(A * X_log)
print(B * X_log)

In [ ]:
log_loss_spec(pred, targets)
#print(pred)

# The pinn class has the exact same interface.

In [ ]:
data_path = '../data/flame_rhs/'
input_prefix = 'react_inputs_*'
output_prefix = 'react_outputs_*'
plotfile_prefix = 'flame_*'
output_dir = 'testing123/'
log_file = output_dir + "log.txt"

In [ ]:
# We first remove the directory we just generated. If you don't do this you'll get an error. This is to protect
# this package from overwriting your data in case one forgets to change the output_dir when training a new model
!rm -r testing123/

In [ ]:
from maestroflame.train import NuclearReactionPinn
nrml = NuclearReactionPinn(data_path, input_prefix, output_prefix, plotfile_prefix,
                output_dir, log_file, DEBUG_MODE=False, DO_PLOTTING=True,
                SAVE_MODEL=True, DO_HYPER_OPTIMIZATION=False)

In [ ]:
# A much more complicated loss function

import torch.nn as nn
from maestroflame.losses import component_loss_f, loss_pinn, rms_weighted_error
from maestroflame.losses import log_loss, loss_mass_fraction, component_loss_f_L1, relative_loss
from maestroflame.losses import derivative_loss_piecewise, signed_loss_function

nnuc=13

def criterion(data, pred, dXdt, actual):
    #I still don't understand this one but i think don does
    #loss1 = rms_weighted_error(pred, actual[:, :nnuc+1], actual[:, :nnuc+1])

    # #difference in state variables vs prediction.
    # loss1 = log_loss(pred, actual[:, :nnuc+1])
    # #physics informed loss
    # loss2 = loss_pinn(data, pred, actual, enuc_fac, enuc_dot_fac, log_option=True)
    # #sum of mass fractions must be 1
    # loss3 = loss_mass_fraction(pred)
    # #relative loss function.
    # loss4 = relative_loss(pred, actual[:, :nnuc+1])


    #difference in state variables vs prediction.
    loss1 = log_loss(pred, actual[:, :nnuc+1])
    #scaled rates (pinn part) This only scales the magnitude of rates
    loss2 = derivative_loss_piecewise(dXdt, actual[:, nnuc+1:], enuc_fac, enuc_dot_fac)
    #L = nn.L1Loss()
    #loss2 = L(dXdt, actual[:, nnuc+1:])

    #here we learn the sign of rates to make up for not doing that in loss2
    loss3 = signed_loss_function(dXdt, actual[:, nnuc+1:])
    #relative loss function. Helps disginguish between same errors of different
    #scales since we're scaling the loss2 so heavily
    loss4 = relative_loss(pred, actual[:, :nnuc+1])
    #sum of mass fractions must be 1
    loss5 = loss_mass_fraction(pred)
    #sum of rates must be 0
    loss6 = loss_rates_mass_frac(dXdt, actual[:, nnuc+1:])

    # loss_arr = [loss1.item(), loss2.item(), loss3.item(), loss4.item(), loss5.item()]
    # return  loss1 + loss2 + loss3  + loss4 + loss5, loss_arr
    loss_arr = [loss1.item(), loss2.item(), loss3.item(), loss5.item()]
    return  loss1 + loss2 + loss3 + loss5, loss_arr

In [ ]:
from maestroflame.networks import Net
import torch.optim as optim
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

num_epochs = 2
model = Net(16, 16, 16, 16, 14)

# get model to cuda if possible
model.to(device=device)
optimizer = optim.Adam(model.parameters(), lr=1e-6)

nrml.train(model,optimizer, num_epochs, criterion)

In [ ]:
# need to put model on cpu for plotting
model.to(device=torch.device("cpu"))

nrml.plot()

In [ ]:
import glob
plots = glob.glob('testing123/*.png')
from IPython.display import Image, display

for plot in plots:
    fig = Image(filename=(plot))
    display(fig)